In [17]:
import pandas as pd # 데이터 핸들링 
import numpy as np 
import matplotlib.pyplot as plt # 데이터 시각화 
%matplotlib inline
import seaborn as sns # 데이터 시각화(고급분석)
import warnings
import json
warnings.filterwarnings("ignore")

In [18]:
data1 = pd.read_csv("ratings_.csv")
data1.info()
data2 = data1.iloc[:100000]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25035538 entries, 0 to 25035537
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ratings_Id     int64  
 1   user_Id        int64  
 2   rating         float64
 3   imdb_movie_Id  int64  
dtypes: float64(1), int64(3)
memory usage: 764.0 MB


In [19]:
data2.head()

,ratings_Id,user_Id,rating,imdb_movie_Id
0,0,1,10.0,110912
1,1,3,10.0,110912
2,2,4,8.0,110912
3,3,5,8.0,110912
4,4,7,8.0,110912


In [7]:
# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = data2.pivot_table('rating', index='user_Id', columns='imdb_movie_Id')

# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

imdb_movie_Id,45152,108394,110912,111495,114787
user_Id,,,,,
1,7.0,10.0,10.0,7.0,10.0
3,0.0,0.0,10.0,0.0,0.0
4,0.0,0.0,8.0,0.0,0.0
5,0.0,0.0,8.0,0.0,0.0
7,0.0,8.0,8.0,10.0,0.0
...,...,...,...,...,...
162533,0.0,0.0,10.0,0.0,0.0
162534,0.0,0.0,9.0,0.0,0.0
162536,0.0,0.0,10.0,0.0,0.0


In [8]:
#사용자-아이템 행렬 데이터를 아이템-사용자 행렬 데이터로 변환

ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

user_Id,1,3,4,5,7,8,10,12,13,14,...,162528,162529,162530,162531,162532,162533,162534,162536,162538,162541
imdb_movie_Id,,,,,,,,,,,,,,,,,,,,,
45152,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108394,10.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110912,10.0,10.0,8.0,8.0,8.0,10.0,9.0,10.0,10.0,10.0,...,6.0,10.0,6.0,10.0,10.0,10.0,9.0,10.0,5.0,7.0


In [9]:
ratings_matrix_T.shape

(5, 83445)

In [10]:
# 아이템간의 코사인 유사도로 아이템 유사도 산출

from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T,ratings_matrix_T)


# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head()

(5, 5)


imdb_movie_Id,45152,108394,110912,111495,114787
imdb_movie_Id,,,,,
45152,1.000000,0.107586,0.151981,0.112595,0.051016
108394,0.107586,1.000000,0.234947,0.709208,0.181827
110912,0.151981,0.234947,1.000000,0.240651,0.108843
111495,0.112595,0.709208,0.240651,1.000000,0.175222
114787,0.051016,0.181827,0.108843,0.175222,1.000000


In [16]:
item_sim_df[108394].sort_values(ascending=False)[1:4]

imdb_movie_Id
111495    0.709208
110912    0.234947
114787    0.181827
Name: 108394, dtype: float64

In [24]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [25]:
# 개인화된 예측 평정
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index = ratings_matrix.index, columns=ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(3)

(83445, 5)


imdb_movie_Id,45152,108394,110912,111495,114787
user_Id,,,,,
1,7.654696,8.902930,9.321652,8.508370,9.552567
3,1.067901,1.051893,5.758965,1.075452,0.717531
4,0.854320,0.841514,4.607172,0.860362,0.574025
